In [1]:
import datetime
import numpy as np
import networkx as nx
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
from itertools import chain
import math
from twittergraph import TwitterGraph as tg
from collections import defaultdict
import random
import re
import json

In [3]:
graph = tg.rt_graph_from_json('/Volumes/pond/Temp/twitter/', 0)

In [4]:
first_split = datetime.datetime(2014, 5, 5)  #shorten the middle time period because it has more edges
second_split = datetime.datetime(2014, 5, 10)

g_0 = tg.tg_by_removing_edges_after_date(graph, first_split)
#tg.remove_degree_zero_nodes(g_0)

g_1 = tg.tg_by_removing_edges_after_date(graph, second_split)
#tg.remove_degree_zero_nodes(g_1)

g_2 = tg.tg_with_tg(graph)
print("New edges in training set: %d" % (g_1.nx_graph.number_of_edges() - g_0.nx_graph.number_of_edges()))
print("New edges in testing set: %d" % (g_2.nx_graph.number_of_edges() - g_1.nx_graph.number_of_edges()))

New edges in training set: 17996
New edges in testing set: 9687


In [5]:
train_pairs = g_0.make_pairs_with_edges(g_1, .5)
test_pairs = g_1.make_pairs_with_edges(g_2, .5)

Found 17996 new edges out of 35993 total pairs
Found 9687 new edges out of 19375 total pairs


In [45]:
g_0.katz_for_pairs(train_pairs, '/Volumes/pond/Temp/SA_RT_ADJ/G_0/SA_RT_ADJ_', max_length=6, beta=0.5)
g_1.katz_for_pairs(test_pairs, '/Volumes/pond/Temp/SA_RT_ADJ/G_1/SA_RT_ADJ_', max_length=6, beta=0.5)

Loaded /Volumes/pond/Temp/SA_RT_ADJ/G_0/SA_RT_ADJ_2.npz


Loaded /Volumes/pond/Temp/SA_RT_ADJ/G_0/SA_RT_ADJ_3.npz


Loaded /Volumes/pond/Temp/SA_RT_ADJ/G_0/SA_RT_ADJ_4.npz


Loaded /Volumes/pond/Temp/SA_RT_ADJ/G_0/SA_RT_ADJ_5.npz


Loaded /Volumes/pond/Temp/SA_RT_ADJ/G_0/SA_RT_ADJ_6.npz


Loaded /Volumes/pond/Temp/SA_RT_ADJ/G_1/SA_RT_ADJ_2.npz


Loaded /Volumes/pond/Temp/SA_RT_ADJ/G_1/SA_RT_ADJ_3.npz


Loaded /Volumes/pond/Temp/SA_RT_ADJ/G_1/SA_RT_ADJ_4.npz


Loaded /Volumes/pond/Temp/SA_RT_ADJ/G_1/SA_RT_ADJ_5.npz


Loaded /Volumes/pond/Temp/SA_RT_ADJ/G_1/SA_RT_ADJ_6.npz


In [53]:
df_train, y_train = g_0.to_dataframe(pairs=train_pairs, sampling=None, label_graph=g_1, min_katz=0, verbose=True)

df_test, y_test = g_1.to_dataframe(pairs=test_pairs, sampling=None, label_graph=g_2, min_katz=0, verbose=True)

Using the pairs you provided...
Precomputing katzes....


35993 pairs checked and 35993 pairs in dataframe
Using the pairs you provided...
Precomputing katzes....


19375 pairs checked and 19375 pairs in dataframe


In [54]:
print("Training on %d new edges out of %d pairs " % (np.sum(y_train), df_train.shape[0]))
print("Testing on %d new edges out of %d pairs" % (np.sum(y_test), df_test.shape[0]))

Training on 17996 new edges out of 35993 pairs 
Testing on 9687 new edges out of 19375 pairs


In [55]:
rf = RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_split=2, random_state=0, n_jobs=-1)
# rf = LinearSVC()
fields = ['katz', 'katz_centrality', 'att', 'adam', 'jac',  'nbrs', 'spl']
x_train = df_train.loc[:, fields]
x_test = df_test.loc[:, fields]

classifier = rf.fit(x_train, y_train)
pred = classifier.predict_proba(x_test)

In [56]:
bin_pred = []
for i in range(len(pred)):
    if pred[i, 1] > .495:
        bin_pred.append(True)
    else:
        bin_pred.append(False)

In [57]:
print(roc_auc_score(y_test, pred[:, 1]))
print(np.sum(bin_pred))

0.673829408163
14415


In [58]:
(pr, re, fs, su) = precision_recall_fscore_support(y_test, bin_pred, average='binary')
print("Precision: %.4f" % pr)
print("Recall: %.4f" % re)
print("F-Score: %.4f" % fs)
# print("Support: %.4f" % su)

Precision: 0.5912
Recall: 0.8797
F-Score: 0.7072


In [59]:
print fields
print rf.feature_importances_

['katz', 'katz_centrality', 'att', 'adam', 'jac', 'nbrs', 'spl']


[ 0.11079573  0.66108854  0.14683587  0.02485823  0.03368835  0.00743596
  0.01529732]


In [60]:
correct = 0
incorrect = 0
correct_edges = 0
incorrect_edges = 0
false_positive = 0
false_negative = 0

for i in range(0,df_test.shape[0]):
    prediction = bin_pred[i]
    actu = y_test[i]
    if prediction == actu:
        correct += 1
        if actu:
            correct_edges += 1
    else:
        incorrect += 1
        if actu:
            incorrect_edges += 1
        else:
            false_positive += 1

print("Correct predictions: %d" % correct)
print("Incorrect predictions: %d\n" % incorrect)

print("%d true positive" % correct_edges)
print("%d false negative" % incorrect_edges)
print("%d false positives" % false_positive)

Correct predictions: 12317
Incorrect predictions: 7058

8522 true positive
1165 false negative
5893 false positives


In [62]:
nx.write_edgelist(g_0.nx_graph,'SA_G0.edgelist' ,data=False)

In [63]:
nx.write_edgelist(g_1.nx_graph,'SA_G1.edgelist', data=False)

In [66]:
embs = pd.read_csv('SA_G0.emd', sep=' ', skiprows=1)

In [67]:
print embs.shape

(12054, 129)


In [70]:
print(g_0.nx_graph.number_of_nodes())
print(g_0.nx_graph.number_of_edges())
zeros = 0
for node in g_0.nx_graph.nodes_iter():
    if g_0.nx_graph.degree(node) == 0:
        zeros += 1
print(zeros)

20718
30510
8663
